In [1]:
import requests
import time
import json
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.avito.ru/rossiya/odezhda_obuv_aksessuary?cd=1&s=104&p={}'

In [28]:
class Offer:
    def __init__(self, title, image, address):
        self.title = title
        self.image = image
        self.address = address
        
    def to_dict(self):
        return {'title': self.title, 'image': self.image, 'address': self.address}

In [31]:
offers = []

for i in range(15):
    page = requests.get(url.format(i + 1))
    soup = BeautifulSoup(page.content, 'html.parser')
    
    items = soup.find_all("div", {"itemtype": "http://schema.org/Product"})
    for item in items:
        details = item.find("img", {"class": "photo-slider-image-1fpZZ"})
        
        try:
            title = details['alt']
            image = details['src']
            address = item.find("span", {"class": "geo-address-9QndR text-text-1PdBw text-size-s-1PUdo"}).find("span").text
        except Exception:
            continue
        
        offers.append(Offer(title, image, address).to_dict())
    time.sleep(7)
    
with open("offers.txt", "w", encoding="utf-8") as f:
    json.dump(offers, f)